In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import mne

from mne.decoding import (
    GeneralizingEstimator,
    LinearModel,
    Scaler,
    SlidingEstimator,
    Vectorizer,
    cross_val_multiscore,
    get_coef,
)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

Defining the triggers and have a look at their timing

In [3]:
epochs = mne.read_epochs("/Volumes/Extreme_SSD/payam_data/Tinreg/epochs/asjt-epo.fif", preload=True)
epochs.pick(picks="eeg")
epochs.drop_bad(reject=dict(eeg=100e-6))
even_ids = epochs.event_id
even_ids.pop("New Segment/", None)
fig, ax = plt.subplots(1, 1, figsize=(10, 4), layout="tight")
mne.viz.plot_events(epochs.events, sfreq=1000, event_id=even_ids, axes=ax)
ax.get_legend().remove()
ax.spines[["right", "top"]].set_visible(False)

Reading /Volumes/Extreme_SSD/payam_data/Tinreg/epochs/asjt-epo.fif ...
    Read a total of 2 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 59) active
        EOG-eeg--0.200-0.200-PCA-02 (1 x 59) active
    Found the data of interest:
        t =    -400.00 ...     490.00 ms
        0 CTF compensation matrices available
Not setting metadata
6000 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
    Rejecting  epoch based on EEG : ['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'TP9', 'TP10', 'T8', 'FT10', 'F4', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FC3', 'TP7', 'POz', 'TP8', 'C6', 'FT8', 'F6', 'AF8', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'TP10', 'FT10', 'F4', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'F6', 'AF8', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'Fp2', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'AF8', '

/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_2929/1991432154.py:7: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  mne.viz.plot_events(epochs.events, sfreq=1000, event_id=even_ids, axes=ax)


In [4]:
## balance trials across the 4 classes in training
rnd_ids = [key for key in even_ids if key.endswith("rndm")]
eps_list = [epochs[rnd_id] for rnd_id in rnd_ids]
mne.epochs.equalize_epoch_counts(eps_list, method="mintime")
epochs_rnd = mne.concatenate_epochs(eps_list)

ord_ids = [key for key in even_ids if key.endswith("or")]
epochs_ord = epochs[ord_ids]

## compute covariance matrix from rnd epochs
cov = mne.compute_covariance(epochs_rnd, tmax=0.0)

## define epochs
ids = range(1, 5) 
epochs_rnd_std = epochs_rnd[[f"f{i}_std_rndm" for i in ids]]
epochs_rnd_tin = epochs_rnd[[f"f{i}_tin_rndm" for i in ids]]

epochs_ord_std = epochs_ord[[f"f{i}_std_or" for i in ids]]
epochs_ord_tin = epochs_ord[[f"f{i}_tin_or" for i in ids]]

del epochs_ord, epochs_rnd

Dropped 11 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
Dropped 23 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22
Dropped 11 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
Dropped 39 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38
Dropped 6 epochs: 6, 21, 24, 43, 49, 50
Dropped 30 epochs: 4, 5, 6, 8, 10, 11, 18, 60, 63, 65, 66, 73, 76, 78, 79, 82, 83, 84, 85, 86, 100, 105, 247, 307, 311, 312, 326, 327, 336, 337
Dropped 37 epochs: 4, 8, 9, 10, 11, 12, 18, 19, 22, 23, 25, 33, 49, 50, 51, 55, 77, 79, 102, 103, 106, 107, 119, 178, 240, 244, 251, 252, 318, 319, 330, 333, 340, 341, 342, 343, 344
Dropped 0 epochs: 
Not setting metadata
2736 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_2929/2087838392.py:5: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_rnd = mne.concatenate_epochs(eps_list)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_2929/2087838392.py:11: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov = mne.compute_covariance(epochs_rnd, tmax=0.0)


    Created an SSP operator (subspace dimension = 2)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 59 -> 57
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 112176
[done]


train clfs on random trials

In [ ]:
n_splits = 5

## create masks 
post_mask = epochs_rnd_std.times >= 0
pre_mask  = epochs_rnd_std.times < 0
X = epochs_rnd_std.get_data()
y = epochs_rnd_std.events[:, 2]

X_post = X[:, :, post_mask]
X_pre = X[:, :, pre_mask]

## define and fit generalization object
clf = make_pipeline(
                    Scaler(epochs_rnd_std.info),
                    Vectorizer(),           
                    LinearModel(LinearDiscriminantAnalysis(solver="svd"))
                    )
gen = GeneralizingEstimator(clf, scoring="accuracy", n_jobs=1, verbose=True)
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

## post -> post
scores_post_post = cross_val_multiscore(gen, X_post, y, cv=cv, n_jobs=1)

## post -> pre
scores_post_pre = []
for train_idx, test_idx in cv.split(X_post, y):
    gen.fit(X_post[train_idx], y[train_idx]) # train on post
    score = gen.score(X_pre[test_idx], y[test_idx]) # test on pre
    scores_post_pre.append(score)

scores_post_pre = np.array(scores_post_pre)

## compute coefficients
coef_filt = get_coef(gen, "filters_", inverse_transform=False) # (n_chs, n_class, n_time)
coef_patt = get_coef(gen, "patterns_", inverse_transform=True)[0] # (n_chs, n_class, n_time)

## save in numpy array  
np.save("", score ...)


## test on ordered tones
X_ord = epochs_ord_std.get_data()
y_ord = epochs_ord_std.events[:, 2]

times_ord = epochs_ord_std.times
post_mask_ord = epochs_ord_std.times >= 0
pre_mask_ord  = epochs_ord_std < 0

X_ord_post = X_ord[:, :, post_mask_ord]
X_ord_pre  = X_ord[:, :, pre_mask_ord]

gen.fit(X_post, y) # train again on random

## scores and coeffs
score_ord_post = gen.score(X_ord_post, y_ord)
score_ord_pre = gen.score(X_ord_pre, y_ord)

coef_filt_ord = get_coef(gen, "filters_", inverse_transform=False) # (n_chs, n_class, n_time)
coef_patt_ord = get_coef(gen, "patterns_", inverse_transform=True)[0] # (n_chs, n_class, n_time)

## save in numpy array  
np.save("", score ...)

now bring it into source space

In [ ]:
def run_source_analysis(coef_patt, epochs):

    evokeds = []
    for i_cls in range(4):
        evokeds.append(
                        mne.EvokedArray(coef_patt[:, i_cls, :],
                                        epochs.info,
                                        tmin=epochs.times[0])
                        )

    noise_cov = compute_covariance(epochs, tmax=0.0)
    kwargs = {
                "subject": "fsaverage",
                "subjects_dir": None
            }

    fs_dir = fetch_fsaverage()
    trans = fs_dir / "bem" / "fsaverage-trans.fif"
    src = fs_dir / "bem" / "fsaverage-ico-5-src.fif"
    bem = fs_dir / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif"

    fwd = make_forward_solution(
                                epochs.info,
                                trans=trans,
                                src=src,
                                bem=bem,
                                meg=False,
                                eeg=True
                                )
    inv = make_inverse_operator(
                                epochs.info,
                                fwd,
                                noise_cov
                                )
    
    for evoked in evokeds:
        stc = apply_inverse(
                            evoked, 
                            inv,
                            lambda2=1.0 / 9.0,
                            method="dSPM",
                            pick_ori="normal"
                            )
    
    del fwd, inv
    
    stc.save("...")

test clfs on ordered trials

In [ ]:
X_ord = epochs_ord_std.get_data()
y_ord = epochs_ord_std.events[:, -1]
times_ord = epochs_ord_std.times

post_mask_ord = (times_ord >= 0) & (times_ord <= 0.5)
pre_mask_ord  = (times_ord >= -0.4) & (times_ord < 0)

X_ord_post = X_ord[:, :, post_mask_ord]
X_ord_pre  = X_ord[:, :, pre_mask_ord]

gen.fit(X_post, y) # from random

score_ord_post = gen.score(X_ord_post, y_ord)
print("Random-post → Ordered-post accuracy:", score_ord_post.mean())

# Test on ordered pre
score_ord_pre = gen.score(X_ord_pre, y_ord)
print("Random-post → Ordered-pre accuracy:", score_ord_pre.mean())

In [ ]:
n_splits = 5

## create masks 
post_mask = epochs_rnd_std.times >= 0
pre_mask  = epochs_rnd_std.times < 0
X = epochs_rnd_std.get_data()
y = epochs_rnd_std.events[:, 2]

X_post = X[:, :, post_mask]
X_pre = X[:, :, pre_mask]

## define and fit generalization object
clf = make_pipeline(
                    Scaler(epochs_rnd_std.info),
                    Vectorizer(),           
                    LinearModel(LinearDiscriminantAnalysis(solver="svd"))
                    )
gen = GeneralizingEstimator(clf, scoring="accuracy", n_jobs=1, verbose=True)
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

## post -> ord
scores_post_post = cross_val_multiscore(gen, X_post, y, cv=cv, n_jobs=1)

## post -> pre
scores_post_pre = []
for train_idx, test_idx in cv.split(X_post, y):
    gen.fit(X_post[train_idx], y[train_idx]) # train on post
    score = gen.score(X_pre[test_idx], y[test_idx]) # test on pre
    scores_post_pre.append(score)

scores_post_pre = np.array(scores_post_pre)

## compute coefficients
coef_filt = get_coef(gen, "filters_", inverse_transform=False) # (n_chs, n_class, n_time)
coef_patt = get_coef(gen, "patterns_", inverse_transform=True)[0] # (n_chs, n_class, n_time)

## save in numpy array  
np.save("", score ...)

plotting

In [ ]:
fig, ax = plt.subplots()
ax.plot(epochs_rnd_std.times, np.diag(avg_scores), label="score")
ax.axhline(0.25, color="k", linestyle="--", label="chance")
ax.set_xlabel("Times")
ax.set_ylabel("AUC")
ax.legend()
ax.axvline(0.0, color="k", linestyle="-")
ax.set_title("Decoding random prestimulus")

In [ ]:
fig, ax = plt.subplots(1, 1)
im = ax.imshow(
    avg_scores,
    interpolation="lanczos",
    origin="lower",
    cmap="RdBu_r",
    extent=epochs_rnd_std.times[[0, -1, 0, -1]],
    vmin=0.0,
    vmax=0.4,
)
ax.set_xlabel("Testing Time (s)")
ax.set_ylabel("Training Time (s)")
ax.set_title("Temporal generalization")
ax.axvline(0, color="k")
ax.axhline(0, color="k")
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("AUC")

In [30]:
import mne

In [ ]:
def decode(subject, saving_dir, epochs_rnd_std, epochs_rnd_tin, epochs_ord_std, epochs_ord_tin):
    
    ###### train clf on random trials
    n_splits = 2
    scores_dir = saving_dir / "scores"
    coeffs_dir = saving_dir / "coeffs"
    stcs_dir = saving_dir / "stcs"
    [sel_dir.mkdir(exist_ok=True) for sel_dir in [scores_dir, coeffs_dir, stcs_dir]]

    labels = ["standard", "tinnitus"]
    for epochs_rnd, epochs_ord, label in zip([epochs_rnd_std, epochs_rnd_tin], [epochs_ord_std, epochs_ord_tin], labels):

        post_mask = epochs_rnd.times >= 0
        pre_mask  = epochs_rnd.times < 0
        X = epochs_rnd.get_data()
        y = epochs_rnd.events[:, 2]

        X_post_rnd = X[:, :, post_mask]
        X_pre_rnd = X[:, :, pre_mask]

        ## define and fit generalization object
        clf = make_pipeline(
                            Scaler(epochs_rnd.info),
                            Vectorizer(),           
                            LinearModel(LinearDiscriminantAnalysis(solver="svd"))
                            )
        gen = GeneralizingEstimator(clf, scoring="accuracy", n_jobs=1, verbose=True)

        ## train post -> test post with fit (to extract weights)
        gen.fit(X_post_rnd, y)
        coef_filt = get_coef(gen, "filters_", inverse_transform=False)
        coef_patt = get_coef(gen, "patterns_", inverse_transform=True)[0] # (n_chs, n_class, n_time)

        np.save(coeffs_dir / f"{subject}_rnd_params_{label}.npy", coef_filt)
        np.save(coeffs_dir / f"{subject}_rnd_patterns_{label}.npy", coef_patt)

        ## train post -> test post with cv
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        scores_post_post = cross_val_multiscore(gen, X_post_rnd, y, cv=cv, n_jobs=1)

        ## train post -> test pre
        scores_post_pre = []
        for train_idx, test_idx in cv.split(X_post_rnd, y):
            gen.fit(X_post_rnd[train_idx], y[train_idx]) # train on post
            score = gen.score(X_pre_rnd[test_idx], y[test_idx]) # test on pre
            scores_post_pre.append(score)
        scores_post_pre = np.array(scores_post_pre)

        ## save scores and coeffs 
        np.save(scores_dir / f"{subject}_rnd_post2post_{label}.npy", scores_post_post)
        np.save(scores_dir / f"{subject}_rnd_post2pre_{label}.npy", scores_post_pre)

        ## source space decoding for random post
        stcs = run_source_analysis(coef_patt, epochs_rnd)
    return stcs